In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Pará - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_PA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Consumo de Cimento (t)
0,2003,3.664692,618.075773,8.348543,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,0.669564,7.776091e+07,4.250430e+06,9.220374,6.851269e+07,1052.310000
1,2004,3.365934,678.161979,8.276683,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,0.671958,7.909171e+07,4.310972e+06,9.281804,6.912419e+07,1193.163000
2,2005,3.175303,747.048967,8.254638,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,0.674195,8.028032e+07,4.348769e+06,9.324669,6.956367e+07,1264.982000
3,2006,3.383776,874.128576,8.168506,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,0.676282,8.143162e+07,4.390026e+06,9.371675,7.001147e+07,1181.617000
4,2007,3.978395,1030.601051,8.144568,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,0.678225,8.248830e+07,4.436626e+06,9.412493,7.039051e+07,1272.803000
5,2008,4.409578,1172.693755,8.116333,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,0.680030,8.343245e+07,4.475828e+06,9.446969,7.069042e+07,1559.491000
6,2009,4.558849,1310.052663,8.148790,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,0.681704,8.418073e+07,4.507563e+06,9.465568,7.086672e+07,1355.544000
7,2010,4.738253,1443.581473,8.112071,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,0.683252,8.675706e+07,4.555784e+06,9.542119,7.180858e+07,1640.682000
8,2011,4.839096,1536.079376,8.099477,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,0.684681,8.890935e+07,4.574157e+06,9.572272,7.235179e+07,1856.176000
9,2012,4.865765,1569.763418,9.368278,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,0.685697,9.072117e+07,4.559887e+06,9.574043,7.265948e+07,2042.910000


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,3.664692,618.075773,8.348543,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,0.669564,7.776091e+07,4.250430e+06,9.220374,6.851269e+07
1,3.365934,678.161979,8.276683,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,0.671958,7.909171e+07,4.310972e+06,9.281804,6.912419e+07
2,3.175303,747.048967,8.254638,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,0.674195,8.028032e+07,4.348769e+06,9.324669,6.956367e+07
3,3.383776,874.128576,8.168506,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,0.676282,8.143162e+07,4.390026e+06,9.371675,7.001147e+07
4,3.978395,1030.601051,8.144568,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,0.678225,8.248830e+07,4.436626e+06,9.412493,7.039051e+07
5,4.409578,1172.693755,8.116333,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,0.680030,8.343245e+07,4.475828e+06,9.446969,7.069042e+07
6,4.558849,1310.052663,8.148790,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,0.681704,8.418073e+07,4.507563e+06,9.465568,7.086672e+07
7,4.738253,1443.581473,8.112071,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,0.683252,8.675706e+07,4.555784e+06,9.542119,7.180858e+07
8,4.839096,1536.079376,8.099477,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,0.684681,8.890935e+07,4.574157e+06,9.572272,7.235179e+07
9,4.865765,1569.763418,9.368278,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,0.685697,9.072117e+07,4.559887e+06,9.574043,7.265948e+07


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1193.163000
1     1264.982000
2     1181.617000
3     1272.803000
4     1559.491000
5     1355.544000
6     1640.682000
7     1856.176000
8     2042.910000
9     2318.608000
10    2041.730714
11    1863.549771
12    1705.680514
13    1564.940000
14    1375.281000
15    1290.928000
16    1444.547000
17    1529.052000
18    1592.624000
19            NaN
Name: Pará - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,-1.138757,-1.758406,-0.752091,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.803855,-1.590257,-1.663515,-2.113181,-1.842041
1,-1.451101,-1.560820,-0.822573,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.508221,-1.391233,-1.059846,-1.650898,-1.502641
2,-1.650401,-1.334294,-0.844196,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.231911,-1.213474,-0.682977,-1.328330,-1.258722
3,-1.432448,-0.916408,-0.928677,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.974158,-1.041295,-0.271603,-0.974596,-1.010184
4,-0.810788,-0.401868,-0.952157,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.734193,-0.883266,0.193050,-0.667428,-0.799809
5,-0.359997,0.065386,-0.979851,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.511248,-0.742067,0.583940,-0.407990,-0.633351
6,-0.203938,0.517073,-0.948016,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.304553,-0.630159,0.900371,-0.268023,-0.535503
7,-0.016376,0.956166,-0.984032,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.113340,-0.244865,1.381186,0.308043,-0.012746
8,0.089054,1.260334,-0.996384,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.063159,0.077014,1.564381,0.534951,0.288749
9,0.116936,1.371099,0.248107,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.188671,0.347976,1.422092,0.548279,0.459520


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1193.163000
1     1264.982000
2     1181.617000
3     1272.803000
4     1559.491000
5     1355.544000
6     1640.682000
7     1856.176000
8     2042.910000
9     2318.608000
10    2041.730714
11    1863.549771
12    1705.680514
13    1564.940000
14    1375.281000
15    1290.928000
16    1444.547000
Name: Pará - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
17,2.159157,-0.855382,0.928663,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.09534,1.061519,-0.73643,1.153715,1.114632


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    1529.052
Name: Pará - Consumo de Cimento (t), dtype: float64

In [12]:
train_input.iloc[-(len(train_input)//5):]

,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
14,1.116838,-0.080187,1.168677,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.388567,1.258699,-1.036694,1.073783,1.189219
15,1.227677,-0.577707,1.074766,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.404430,1.284737,-0.912968,1.157312,1.234960
16,1.694083,-0.841081,1.042191,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,1.275773,1.262555,-0.833260,1.268272,1.275259


In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [14]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [16]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2539362795, 2058758878, 348406884, 3873581357, 4087915274, 2131939203, 2965439848, 3101543867, 1315381814, 1810714232, 2948100624, 3567693340, 3097014073, 3335508923, 3129234813, 2591514628, 2775073796, 1504539495, 907213133, 3551168535, 934032029, 1573490771, 3531864410, 642391170, 3251478021, 2831692417, 1468022336, 2191515034, 3577818696, 1601273312, 1126231614, 3774979269, 3593818387, 495196798, 1782164358, 1270883649, 1744878310, 4239282498, 2667513848, 2729406556, 1820078708, 2897464103, 1521317269, 3477671132, 2807978250, 218127125, 3799782299, 3756098820, 1731222887, 2643772616, 1694293878, 1314822294, 2612565314, 3818963768, 758297399, 944290291, 588802792, 4005292834, 3788302003, 3911799518, 4071379222, 3201360018, 2037756183, 621885668, 1728331865, 1189829364, 35413973, 3460889848, 4115007882, 119291425, 3662200797, 2760333752, 3242305432, 698615451, 2833844308, 527590559, 3899681364, 132221914, 1466335011, 2970944098, 2736331612, 4201335449, 2134616688, 2207472009, 4125952

val_loss: 103163.8359375


Step: 86 ___________________________________________
val_loss: 15686.65234375


Step: 87 ___________________________________________
val_loss: 2175.794921875


Step: 88 ___________________________________________
val_loss: 41927.69140625


Step: 89 ___________________________________________
val_loss: 11720.2685546875


Step: 90 ___________________________________________
val_loss: 92681.3828125


Step: 91 ___________________________________________
val_loss: 53529.28125


Step: 92 ___________________________________________
val_loss: 25301.708984375


Step: 93 ___________________________________________
val_loss: 22887.28125


Step: 94 ___________________________________________
val_loss: 92794.6484375


Step: 95 ___________________________________________
val_loss: 55399.41796875


Step: 96 ___________________________________________
val_loss: 45392.203125


Step: 97 ___________________________________________
val_loss: 10288.9462890625


Step: 98 _________

In [17]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 437ms/step - loss: 2625822.5000 - val_loss: 1770910.3750
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 2511578.0000 - val_loss: 763392.3125
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 2057516.7500 - val_loss: 135301.6406
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 2922762.0000 - val_loss: 1288267.1250
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 2215555.0000 - val_loss: 2233894.0000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 2208065.0000 - val_loss: 2821148.7500
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 2465297.0000 - val_loss: 2816880.2500
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 2259680.5000 - val_loss: 896198.2500
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 2215650.0000 - val_loss: 2052426

1/1 [==============================] - 0s 13ms/step - loss: 488105.3438 - val_loss: 207908.7969
Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 482326.0625 - val_loss: 227157.4219
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 479824.6250 - val_loss: 198787.0781
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 463598.6562 - val_loss: 136106.1094
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 441826.0000 - val_loss: 195194.3281
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 441304.0938 - val_loss: 179497.9531
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 414455.9062 - val_loss: 157983.7656
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 392516.2500 - val_loss: 202455.3281
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 407458.0000 - val_loss: 151969.5469
Epoch 83/10000
1

Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 164687.6406 - val_loss: 10768.7236
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 136922.3125 - val_loss: 6161.8457
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 111232.7266 - val_loss: 9536.3047
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 134303.4375 - val_loss: 10690.7373
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 129748.5703 - val_loss: 14279.5078
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 109884.3359 - val_loss: 15664.4844
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 107636.2812 - val_loss: 5019.7520
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 114209.1797 - val_loss: 4350.5215
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 121083.9844 - val_loss: 4955.8999
Epoch 

1/1 [==============================] - 0s 13ms/step - loss: 93970.1406 - val_loss: 30126.1309
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 60733.4922 - val_loss: 27907.6934
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 80817.1094 - val_loss: 34775.1250
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 91007.8750 - val_loss: 26162.5137
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 91670.9297 - val_loss: 26214.0391
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 90838.5625 - val_loss: 26951.4980
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 86223.8906 - val_loss: 34763.2500
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 54606.9141 - val_loss: 23295.8984
Epoch 230/10000
1/1 [==============================] - 0s 13ms/step - loss: 67468.0000 - val_loss: 24123.1309
Epoch 231/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 58291.5703 - val_loss: 48154.6719
Epoch 297/10000
1/1 [==============================] - 0s 13ms/step - loss: 46692.1680 - val_loss: 47578.8633
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 62388.7070 - val_loss: 34761.6992
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 63502.1328 - val_loss: 67727.2266
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 58864.1992 - val_loss: 43701.1680
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 41585.1641 - val_loss: 49719.4023
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 61314.6016 - val_loss: 56454.0977
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 42698.6133 - val_loss: 38009.8477
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 43269.1562 - val_loss: 45805.8633
Epoch 305/10000
1/1 [=====

...

Epoch 1556/10000
1/1 [==============================] - 0s 14ms/step - loss: 29884.5957 - val_loss: 51202.0156
Epoch 1557/10000
1/1 [==============================] - 0s 13ms/step - loss: 26831.2148 - val_loss: 55688.8711
Epoch 1558/10000
1/1 [==============================] - 0s 14ms/step - loss: 31880.1328 - val_loss: 57287.9219
Epoch 1559/10000
1/1 [==============================] - 0s 13ms/step - loss: 27151.6934 - val_loss: 54722.9258
Epoch 1560/10000
1/1 [==============================] - 0s 13ms/step - loss: 24882.1094 - val_loss: 46209.1445
Epoch 1561/10000
1/1 [==============================] - 0s 13ms/step - loss: 26734.1406 - val_loss: 56606.4531
Epoch 1562/10000
1/1 [==============================] - 0s 14ms/step - loss: 20853.2109 - val_loss: 45991.6211
Epoch 1563/10000
1/1 [==============================] - 0s 13ms/step - loss: 26806.1797 - val_loss: 58959.2383
Epoch 1564/10000
1/1 [==============================] - 0s 13ms/step - loss: 26060.1211 - val_loss: 53151.5430
E

Epoch 1630/10000
1/1 [==============================] - 0s 13ms/step - loss: 41212.5938 - val_loss: 57537.9180
Epoch 1631/10000
1/1 [==============================] - 0s 13ms/step - loss: 42010.9180 - val_loss: 62617.4062
Epoch 1632/10000
1/1 [==============================] - 0s 13ms/step - loss: 33604.3750 - val_loss: 59504.6680
Epoch 1633/10000
1/1 [==============================] - 0s 13ms/step - loss: 29956.9336 - val_loss: 63929.0820
Epoch 1634/10000
1/1 [==============================] - 0s 13ms/step - loss: 27732.2539 - val_loss: 64819.7461
Epoch 1635/10000
1/1 [==============================] - 0s 13ms/step - loss: 37866.0938 - val_loss: 66765.6797
Epoch 1636/10000
1/1 [==============================] - 0s 13ms/step - loss: 31580.6992 - val_loss: 64119.9219
Epoch 1637/10000
1/1 [==============================] - 0s 13ms/step - loss: 30971.4746 - val_loss: 67458.8125
Epoch 1638/10000
1/1 [==============================] - 0s 13ms/step - loss: 30730.7188 - val_loss: 66487.1562
E

Epoch 1704/10000
1/1 [==============================] - 0s 13ms/step - loss: 37438.4453 - val_loss: 54468.6289
Epoch 1705/10000
1/1 [==============================] - 0s 13ms/step - loss: 33838.8906 - val_loss: 51670.1250
Epoch 1706/10000
1/1 [==============================] - 0s 13ms/step - loss: 32626.3828 - val_loss: 53041.6250
Epoch 1707/10000
1/1 [==============================] - 0s 13ms/step - loss: 40569.8672 - val_loss: 63695.3086
Epoch 1708/10000
1/1 [==============================] - 0s 14ms/step - loss: 28221.9980 - val_loss: 52384.4023
Epoch 1709/10000
1/1 [==============================] - 0s 14ms/step - loss: 25760.0469 - val_loss: 50497.5664
Epoch 1710/10000
1/1 [==============================] - 0s 14ms/step - loss: 27350.7949 - val_loss: 55387.6523
Epoch 1711/10000
1/1 [==============================] - 0s 14ms/step - loss: 29837.9082 - val_loss: 58128.1758
Epoch 1712/10000
1/1 [==============================] - 0s 14ms/step - loss: 25606.9531 - val_loss: 48374.2188
E

Epoch 1778/10000
1/1 [==============================] - 0s 14ms/step - loss: 22767.5723 - val_loss: 40592.5195
Epoch 1779/10000
1/1 [==============================] - 0s 14ms/step - loss: 29798.4453 - val_loss: 34607.8203
Epoch 1780/10000
1/1 [==============================] - 0s 13ms/step - loss: 21939.5508 - val_loss: 40163.9688
Epoch 1781/10000
1/1 [==============================] - 0s 13ms/step - loss: 29840.5117 - val_loss: 34537.8789
Epoch 1782/10000
1/1 [==============================] - 0s 13ms/step - loss: 30580.0957 - val_loss: 38292.7773
Epoch 1783/10000
1/1 [==============================] - 0s 14ms/step - loss: 21758.3750 - val_loss: 38830.5039
Epoch 1784/10000
1/1 [==============================] - 0s 13ms/step - loss: 24255.0762 - val_loss: 37717.5117
Epoch 1785/10000
1/1 [==============================] - 0s 13ms/step - loss: 14389.1992 - val_loss: 31466.8984
Epoch 1786/10000
1/1 [==============================] - 0s 13ms/step - loss: 21679.7969 - val_loss: 40548.3477
E

Epoch 1852/10000
1/1 [==============================] - 0s 13ms/step - loss: 23421.8984 - val_loss: 36716.3672
Epoch 1853/10000
1/1 [==============================] - 0s 13ms/step - loss: 15801.0254 - val_loss: 44387.2812
Epoch 1854/10000
1/1 [==============================] - 0s 13ms/step - loss: 15202.5059 - val_loss: 45859.7812
Epoch 1855/10000
1/1 [==============================] - 0s 13ms/step - loss: 14869.9453 - val_loss: 46113.6992
Epoch 1856/10000
1/1 [==============================] - 0s 13ms/step - loss: 24866.8047 - val_loss: 41557.8320
Epoch 1857/10000
1/1 [==============================] - 0s 13ms/step - loss: 23007.4863 - val_loss: 38871.0859
Epoch 1858/10000
1/1 [==============================] - 0s 14ms/step - loss: 13941.5322 - val_loss: 35871.5703
Epoch 1859/10000
1/1 [==============================] - 0s 13ms/step - loss: 20257.9746 - val_loss: 38271.2578
Epoch 1860/10000
1/1 [==============================] - 0s 13ms/step - loss: 19565.7656 - val_loss: 39630.0273
E

In [18]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[1529.052](test_target) - [[1424.3464]](prediction) = 104.70556445312491


In [19]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [20]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,-0.434695,-1.283831,1.884449,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.635956,-1.529973,-1.640359,-1.656369,-1.638970
1,-0.965518,-1.073189,0.995176,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.102232,-1.041349,-1.016036,-1.031493,-1.015819
2,-1.304224,-0.831695,0.722359,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.603396,-0.604930,-0.626272,-0.595472,-0.567975
3,-0.933818,-0.386198,-0.343539,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.138061,-0.182213,-0.200823,-0.117323,-0.111649
4,0.122681,0.162340,-0.639777,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.295160,0.205764,0.279729,0.297881,0.274608
5,0.888792,0.660468,-0.989192,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,0.697655,0.552423,0.683993,0.648569,0.580233
6,1.154012,1.142000,-0.587533,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,1.070812,0.827169,1.011251,0.837764,0.759885
7,1.472770,1.610106,-1.041943,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,1.416018,1.773109,1.508517,1.616443,1.719687


train_target:


,Pará - Consumo de Cimento (t)
0,1193.163
1,1264.982
2,1181.617
3,1272.803
4,1559.491
5,1355.544
6,1640.682
7,1856.176


test_input:


...

,Pará - Valor Cimento Portland,Pará - Produção de Cimento (t),Pará - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
17,2.159157,-0.855382,0.928663,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.09534,1.061519,-0.73643,1.153715,1.114632


test_target:


,Pará - Consumo de Cimento (t)
17,1529.052


1/1 [==============================] - 0s 34ms/step
Error: 104.70556445312491




[1638.4476318359375,
 1748.35107421875,
 2059.586181640625,
 2158.025146484375,
 2006.65185546875,
 1919.854248046875,
 1785.969482421875,
 1688.5673828125,
 1625.736328125,
 1424.346435546875]

In [21]:
display(targets)
display(predictions)

[2042.91,
 2318.608,
 2041.730714285714,
 1863.549771428571,
 1705.6805142857138,
 1564.94,
 1375.281,
 1290.928,
 1444.547,
 1529.052]

[1638.4476318359375,
 1748.35107421875,
 2059.586181640625,
 2158.025146484375,
 2006.65185546875,
 1919.854248046875,
 1785.969482421875,
 1688.5673828125,
 1625.736328125,
 1424.346435546875]

In [22]:
mae = mean_absolute_error(predictions, targets)
mae

303.7158483398439

In [23]:
porcentage = mae/np.mean(targets)
porcentage

0.1768130841723428